In [1]:
import torch
import pandas as pd
import numpy as np
import torch.nn as nn
from modules import UNet_conditional
from diffusion import *
from utils import *
from torch.amp import autocast

In [2]:
def predict(model,
            sampler,
            test_dl,
            device,
            n_samples=1):
    """
    Return predictions
    """
    x_real = []
    predictions = []

    model.eval()

    with torch.no_grad():
        for i, data in enumerate(tqdm(test_dl, desc="Testing loop")):
            vectors = data['data'].to(device)
            settings = data['settings'].to(device)

            pred = sampler.ddim_sample_loop(model=model,
                                            y=settings,
                                            cfg_scale=1,
                                            device=device,
                                            eta=1,
                                            n=n_samples
                                            )

            x_real.extend(vectors.cpu().tolist() * n_samples)
            predictions.append(pred.cpu().tolist())

    return x_real, predictions
    

In [3]:
def evaluate(model,
             sampler,
             device,
             test_csv_path,
             n_samples=1):
    """
    Evaluate predictions
    """
    # Load the test dataset
    x_test, y_test = get_data(test_csv_path)

    test_dataset = CustomDataset(x_test, y_test)
    test_dataloader = DataLoader(test_dataset,
                                 batch_size=1,
                                 shuffle=False)

    x_real, predictions = predict(model,
                                  sampler,
                                  test_dataloader,
                                  device=device,
                                  n_samples=n_samples)

    # intesities are normalized
    #x_real = [[x * 3925 for x in row] for row in x_real]

    return x_real, predictions

In [4]:
path = "models/test/ema_ckpt.pt"
print("Loading: ", path)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

model = UNet_conditional(length=1024,
                         feat_num=3,
                         device=device).to(device)
ckpt = torch.load(path, map_location=device, weights_only=True)
model.load_state_dict(ckpt)

sampler = SpacedDiffusion(beta_start=1e-4,
                          beta_end=0.02,
                          noise_steps=1000,
                          section_counts=[40],
                          length=1024,
                          device=device,
                          rescale_timesteps=False)

x_real, predictions = evaluate(model,
                      sampler,
                      device,
                      "../data/test_data.csv")
x_real = np.array(x_real)
predictions = np.array(predictions)

Loading:  models/test/ema_ckpt.pt
Using device: cuda


Testing loop: 100%|███████████████████████████████████████████████| 1567/1567 [16:19<00:00,  1.60it/s]


In [5]:
mse = np.mean((x_real - predictions) ** 2)
print(f"Mean test mse error: {mse}")

Mean test mse error: 0.001050625674436355


In [7]:
x_real

array([[0.00061579, 0.        , 0.        , ..., 0.00181409, 0.00107043,
        0.00106402],
       [0.00087057, 0.        , 0.        , ..., 0.00079498, 0.00183476,
        0.00029969],
       [0.00010624, 0.00054038, 0.        , ..., 0.        , 0.00081565,
        0.00106402],
       ...,
       [0.00112534, 0.        , 0.        , ..., 0.00054021, 0.00056087,
        0.00055447],
       [0.00087057, 0.        , 0.        , ..., 0.00155931, 0.00157998,
        0.00029969],
       [0.        , 0.        , 0.        , ..., 0.00079498, 0.0013252 ,
        0.00029969]])

In [8]:
predictions

array([[2.55865743e-05, 1.61902048e-04, 0.00000000e+00, ...,
        3.96288466e-04, 5.51547157e-04, 1.03841443e-03],
       [3.41543928e-05, 0.00000000e+00, 0.00000000e+00, ...,
        4.41147014e-04, 9.24172346e-04, 1.27354078e-03],
       [2.53366306e-04, 4.54764813e-04, 4.62526223e-05, ...,
        8.21787864e-04, 6.15682337e-04, 9.04959743e-04],
       ...,
       [0.00000000e+00, 0.00000000e+00, 1.30567933e-04, ...,
        1.28671248e-03, 2.38519441e-03, 7.88288482e-04],
       [0.00000000e+00, 0.00000000e+00, 2.98226485e-04, ...,
        0.00000000e+00, 0.00000000e+00, 6.71108719e-04],
       [0.00000000e+00, 1.58078969e-04, 4.88881487e-04, ...,
        1.30645093e-03, 1.20338053e-03, 7.56176654e-04]])

In [9]:
x_real_str = [','.join(map(str, row)) for row in x_real]
preds_str = [','.join(map(str, row)) for row in predictions]

df = pd.DataFrame({'x_real': x_real_str, 'predictions': preds_str})
df.to_csv('results/predictions/preds_test_0.csv', index=False)

In [10]:
df.head()

,x_real,predictions
0,"0.0006157915922813118,0.0,0.0,0.00149204465560...","2.5586574338376522e-05,0.0001619020476937294,0..."
1,"0.0008705681539140642,0.0,0.0,0.00174682121723...","3.4154392778873444e-05,0.0,0.0,0.0016265576705..."
2,"0.0001062385126715526,0.0005403818213380873,0....","0.00025336630642414093,0.00045476481318473816,..."
3,"0.0008705681539140642,3.082872717641294e-05,0....","0.0001150940079241991,0.0,0.000650710426270961..."
4,"0.0003610150597523898,0.0,0.0,0.00123726809397...","0.0,0.001548011670820415,0.04340150207281113,0..."


In [11]:
# Find indices and values greater than 0.5
indices = np.where(np.any(x_real > 0.5, axis=1))[0]
filtered_values = [x_real[idx][x_real[idx] > 0.5] for idx in indices]
filtered_predictions = [predictions[idx][x_real[idx] > 0.5] for idx in indices]

# Flatten results and take the first 50
flat_filtered_values = np.concatenate(filtered_values)[:50]
flat_filtered_predictions = np.concatenate(filtered_predictions)[:50]

# Print the results side by side
print("Filtered Values (greater than 0.5) from x_real and corresponding Predictions:")
for value, prediction in zip(flat_filtered_values, flat_filtered_predictions):
    print(f"{value:.5f}\t\t\t{prediction:.5f}")

Filtered Values (greater than 0.5) from x_real and corresponding Predictions:
0.51206			0.50578
0.51917			0.50961
0.53315			0.52620
0.50054			0.48889
0.51451			0.49924
0.57714			0.56252
0.62085			0.60734
0.67306			0.65777
0.75889			0.73940
0.82681			0.81006
0.89323			0.87176
0.94675			0.92372
0.98621			0.96485
1.00000			0.99850
1.00000			0.98095
1.00000			0.99556
0.98376			0.96781
0.91490			0.90392
0.86643			0.85813
0.81041			0.80300
0.76278			0.75437
0.68614			0.68407
0.66184			0.65592
0.57601			0.57123
0.53988			0.53350
0.50519			0.46873
0.51203			0.47604
0.53162			0.49479
0.50025			0.46325
0.54682			0.50994
0.58773			0.54683
0.63280			0.58922
0.71023			0.66183
0.77866			0.72333
0.83973			0.78108
0.89197			0.83113
0.93576			0.87069
0.97427			0.91007
0.95795			0.89386
0.98165			0.91487
0.95217			0.88822
0.89197			0.83341
0.85242			0.79351
0.80047			0.74384
0.75615			0.70114
0.69047			0.63641
0.66184			0.61125
0.57448			0.53297
0.53402			0.49819
0.50635			0.46179


In [12]:
# Find indices where predictions > 0.8 and corresponding x_real < 0.5
indices = np.where((predictions > 0.8) & (x_real < 0.1))

# Extract filtered values from x_real and predictions
filtered_x_real = x_real[indices]
filtered_predictions = predictions[indices]

# Print the results side by side
print("Filtered x_real (values < 0.1) and corresponding Predictions (values > 0.8):")
for x_val, pred_val in zip(filtered_x_real.flatten(), filtered_predictions.flatten()):
    print(f"{x_val:.5f}\t\t\t{pred_val:.5f}")

Filtered x_real (values < 0.1) and corresponding Predictions (values > 0.8):
0.09600			0.90357
0.08123			0.90569
0.06909			0.86183
0.05302			0.94673
0.02904			1.00000
0.02426			0.92044
